In [1]:
import pandas as pd

In [2]:
df_AOD=pd.read_csv("AOD.csv")
df_temp=pd.read_csv("Temperature.csv")
df_RH=pd.read_csv("Relative_humidity.csv")
df_PM=pd.read_csv("PM2.5.csv")

In [3]:
df_AOD.head()

,lat,lon,latv,lonv,Aerosol_Optical_Depth(Land),lat_bnds,lon_bnds
0,-82.5,-177.5,0,0,NaN,-83.0,-178.0
1,-82.5,-177.5,0,1,NaN,-83.0,-177.0
2,-82.5,-177.5,1,0,NaN,-82.0,-178.0
3,-82.5,-177.5,1,1,NaN,-82.0,-177.0
4,-82.5,-176.5,0,0,NaN,-83.0,-177.0


In [4]:
df_AOD=df_AOD.drop(['lat', 'lon', 'latv', 'lonv'], axis=1, inplace=False)
df_AOD.head()

,Aerosol_Optical_Depth(Land),lat_bnds,lon_bnds
0,NaN,-83.0,-178.0
1,NaN,-83.0,-177.0
2,NaN,-82.0,-178.0
3,NaN,-82.0,-177.0
4,NaN,-83.0,-177.0


In [5]:
df_PM.head()

,lat,lon,latv,lonv,PM2.5,lat_bnds,lon_bnds
0,-82.5,-177.500,0,0,7.630605e-10,-82.75,-177.8125
1,-82.5,-177.500,0,1,7.630605e-10,-82.75,-177.1875
2,-82.5,-177.500,1,0,7.630605e-10,-82.25,-177.8125
3,-82.5,-177.500,1,1,7.630605e-10,-82.25,-177.1875
4,-82.5,-176.875,0,0,7.596716e-10,-82.75,-177.1875


In [6]:
df_RH=df_RH.drop(['lat', 'lon', 'latv', 'lonv'], axis=1, inplace=False )
df_PM=df_PM.drop(['lat', 'lon', 'latv', 'lonv'], axis=1, inplace=False )
df_temp=df_temp.drop(['lat', 'lon', 'latv', 'lonv'], axis=1, inplace=False )
# df_AOD = df_AOD.round()
df_AOD = df_AOD.round({'lat_bnds': 0, 'lon_bnds':0})
df_PM = df_PM.round({'lat_bnds': 0, 'lon_bnds':0}) 
df_RH = df_RH.round({'lat_bnds': 0, 'lon_bnds':0}) 
df_temp = df_temp.round({'lat_bnds': 0, 'lon_bnds':0})
df_RH.to_csv("outputRH.csv", index=False)
df_PM.to_csv("outputPM.csv", index=False)
df_temp.to_csv("outputtemp.csv", index=False)
df_AOD1= df_AOD.interpolate(method='polynomial', order=2).bfill()
df_AOD1.to_csv("outputAOD1.csv", index=False)

In [7]:
df_PM.head(10)

,PM2.5,lat_bnds,lon_bnds
0,7.630605e-10,-83.0,-178.0
1,7.630605e-10,-83.0,-177.0
2,7.630605e-10,-82.0,-178.0
3,7.630605e-10,-82.0,-177.0
4,7.596716e-10,-83.0,-177.0
5,7.596716e-10,-83.0,-177.0
6,7.596716e-10,-82.0,-177.0
7,7.596716e-10,-82.0,-177.0
8,7.596385e-10,-83.0,-177.0
9,7.596385e-10,-83.0,-176.0


In [8]:
common_values = set(df_AOD1['lat_bnds']) & set(df_temp['lat_bnds'])

In [9]:
print(len(common_values))

172


In [10]:
print(df_AOD1.shape, df_temp.shape)

(240084, 3) (240084, 3)


In [11]:
# df_inner1=pd.merge(df_AOD1, df_temp, on=['lat_bnds','lon_bnds'], how='inner')
# df_inner2=pd.merge(df_PM, df_RH, on=['lat_bnds','lon_bnds'], how='inner')


In [12]:
# Remove duplicates from all DataFrames based on lat_bnds and lon_bnds
df_AOD1 = df_AOD1.drop_duplicates(subset=['lat_bnds', 'lon_bnds']).reset_index(drop=True)
df_temp = df_temp.drop_duplicates(subset=['lat_bnds', 'lon_bnds']).reset_index(drop=True)
df_PM = df_PM.drop_duplicates(subset=['lat_bnds', 'lon_bnds']).reset_index(drop=True)
df_RH = df_RH.drop_duplicates(subset=['lat_bnds', 'lon_bnds']).reset_index(drop=True)

In [13]:
# Merge DataFrames after removing duplicates
df_inner1 = df_AOD1.merge(df_temp, on=['lat_bnds', 'lon_bnds'], how='inner')
df_inner2 = df_PM.merge(df_RH, on=['lat_bnds', 'lon_bnds'], how='inner')
df_final = pd.merge(df_inner1, df_inner2, on=['lat_bnds', 'lon_bnds'], how='inner')

In [14]:
# Validate the shape of the merged DataFrame
print('Shape of df_AOD1:', df_AOD1.shape)
print('Shape of df_temp:', df_temp.shape)
print('Shape of df_PM:', df_PM.shape)
print('Shape of df_RH:', df_RH.shape)
print('Shape of df_inner1:', df_inner1.shape)
print('Shape of df_inner2:', df_inner2.shape)
print('Shape of df_final:', df_final.shape)

Shape of df_AOD1: (60544, 3)
Shape of df_temp: (60544, 3)
Shape of df_PM: (60544, 3)
Shape of df_RH: (60544, 3)
Shape of df_inner1: (60544, 4)
Shape of df_inner2: (60544, 4)
Shape of df_final: (60544, 6)


In [15]:
# df_inner1=df_AOD1.merge(df_temp, on=['lat_bnds','lon_bnds'], how='inner')
# df_inner2=df_PM.merge(df_RH, on=['lat_bnds','lon_bnds'], how='inner')

In [16]:
# df_final=pd.merge(df_inner1, df_inner2, on=['lat_bnds','lon_bnds'], how='inner')

In [17]:
# print(df_inner1.shape, df_inner2.shape, df_final.shape)

In [18]:
df_final.head()

,Aerosol_Optical_Depth(Land),lat_bnds,lon_bnds,Average_Temperature,PM2.5,Relative_humidity
0,0.823324,-83.0,-178.0,243.11458,7.630605e-10,78.6250
1,0.823324,-83.0,-177.0,243.11458,7.630605e-10,78.6250
2,0.823324,-82.0,-178.0,243.11458,7.630605e-10,78.6250
3,0.823324,-82.0,-177.0,243.11458,7.630605e-10,78.6250
4,0.823324,-83.0,-176.0,238.50000,7.596385e-10,79.3125


In [19]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60544 entries, 0 to 60543
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Aerosol_Optical_Depth(Land)  58355 non-null  float64
 1   lat_bnds                     60544 non-null  float64
 2   lon_bnds                     60544 non-null  float64
 3   Average_Temperature          49881 non-null  float64
 4   PM2.5                        60544 non-null  float64
 5   Relative_humidity            49462 non-null  float64
dtypes: float64(6)
memory usage: 2.8 MB


In [20]:
df_inner1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60544 entries, 0 to 60543
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Aerosol_Optical_Depth(Land)  58355 non-null  float64
 1   lat_bnds                     60544 non-null  float64
 2   lon_bnds                     60544 non-null  float64
 3   Average_Temperature          49881 non-null  float64
dtypes: float64(4)
memory usage: 1.8 MB


In [21]:
df_inner2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60544 entries, 0 to 60543
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PM2.5              60544 non-null  float64
 1   lat_bnds           60544 non-null  float64
 2   lon_bnds           60544 non-null  float64
 3   Relative_humidity  49462 non-null  float64
dtypes: float64(4)
memory usage: 1.8 MB


In [22]:
df_final = df_final.loc[df_final['Aerosol_Optical_Depth(Land)'] != 0]

In [23]:
df_final = df_final.loc[df_final['PM2.5'] != 0]

In [24]:
# df_data1=df_final[['lat_bnds','lon_bnds', 'Aerosol_Optical_Depth(Land)', 'PM2.5', 'Average_Temperature' , 'Relative_humidity']].copy()

In [25]:
# Rearrange columns in the specified order
df_final = df_final[['lat_bnds', 'lon_bnds', 'Aerosol_Optical_Depth(Land)', 'Average_Temperature', 'Relative_humidity', 'PM2.5']]
df_final=df_final.dropna()

In [26]:
df_final.to_csv('Practical_Problem_data_set1.csv', index=False)